In [32]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [33]:
url = "https://en.wikipedia.org/wiki/List_of_ONE_Championship_events#Events"

soup = BeautifulSoup(
    requests.get(url).text,
    "html.parser"
)

In [34]:
table = soup.find_all("table", class_ = "sortable")[0]
table_rows = table.find_all("tr")[1:]

In [35]:
anchors = [_.find_all("a") for _ in table_rows if len(_.find_all("a"))>0]
cols = ["link", "name"]

In [36]:
raw_df = pd.read_csv(r"C:\Development\ultimateNakMuay\data\raw\wiki_events_onefc.csv")
raw_df.head(3)

,#,Event,Date,Venue,Location,Attendance
0,253,ONE Fight Night 13: Allazov vs. Grigorian,"August 5, 2023",Lumpinee Boxing Stadium,"Bangkok, Thailand",NaN
1,252,ONE Fight Night 12,"July 15, 2023",Lumpinee Boxing Stadium,"Bangkok, Thailand",NaN
2,251,ONE Friday Fights 22: Bhullar vs. Malykhin,"June 23, 2023",Lumpinee Boxing Stadium,"Bangkok, Thailand",NaN


In [37]:
links_df = pd.DataFrame([(anchor[0].get("href"), anchor[0].text) for anchor in anchors], columns=cols)

In [38]:
clean_link = lambda x: f"https://en.wikipedia.org/{x.split('#')[0]}"
links_df["link_clean"] = links_df["link"].apply(clean_link)
links = links_df["link_clean"].value_counts().reset_index().iloc[:,0]
links = links.to_list()
links = list(set([link for link in links if "one_" in link.lower()]))

In [39]:
def scrapeTableM1(url):
    soup = BeautifulSoup(requests.get(url).text)

    events = soup.find_all(
        lambda tag: tag.name == "span" and 
        ("one " in tag.text.lower() or "fight card" in tag.text.lower() or "results" in tag.text.lower())
        and tag.get("class") == ["mw-headline"]
        )
    
    dfs = []

    for event in events:
        df = pd.read_html(str(event.find_next("table", class_ = "toccolours")))[0]
        if df.empty:
            df = pd.read_html(str(event.find_next("table", class_ = "wikitable")))[0]
        df = df.assign(event_name = event.text)
        df = df.assign(event_url = url)
        dfs.append(df)

    return pd.concat(dfs, ignore_index=True)

In [40]:
m1_results = []
failed_links = []
for link in links:
    try:
        m1_results.append(scrapeTableM1(link))
    except:
        failed_links.append(link)

In [41]:
def scrapeTableM2(url):
    soup = BeautifulSoup(requests.get(url).text)
    headers = soup.find_all(lambda tag: tag.name == "h2")
    headers = [_ for _ in headers if _.find(lambda tag: tag.name == "span" and "one" in tag.text.lower()) != None]
    headers = [
        {header: header.find_next(lambda tag: tag.name=="h3" and tag.find("span", id = "Results")!=None)}
        for header in headers
        ]
    headers = [header for header in headers if header[list(header)[0]] != None]
    dfs = []
    for header in headers:
        df = pd.read_html(str(list(header.values())[0].findNext("table")))[0]
        df = df.assign(event_name = list(header)[0].text.replace("[edit]", "").strip())
        dfs.append(df)

    return pd.concat(dfs, ignore_index=True)

In [42]:
m2_results = []
for link in failed_links:
    try:
        m2_results.append(scrapeTableM2(link))
    except:
        print(link)

In [43]:
set([len(df.columns) for df in m2_results])

{9}

In [44]:
for result in m1_results:
    result.columns = [_[0] if _[1]=="" else _[1] for  _ in result.columns]

In [45]:
for i, df in enumerate(m1_results):
    if len(df.columns) == 18:
        m1_results[i] = df.iloc[:,:-8]

In [46]:
set(len(df.columns) for df in m2_results)

{9}

In [47]:
for result in m2_results:
    result.columns = [_[0] if _[1]=="" else _[1] for  _ in result.columns]

In [48]:
dfm1 = pd.concat(m1_results, ignore_index=True)
dfm2 = pd.concat(m2_results, ignore_index=True)

In [65]:
failed_links

['https://en.wikipedia.org//wiki/ONE_Friday_Fights_16',
 'https://en.wikipedia.org//wiki/2021_in_ONE_Championship',
 'https://en.wikipedia.org//wiki/2023_in_ONE_Championship',
 'https://en.wikipedia.org//wiki/ONE_Friday_Fights_15',
 'https://en.wikipedia.org//wiki/2017_in_ONE_Championship',
 'https://en.wikipedia.org//wiki/ONE_Friday_Fights_14',
 'https://en.wikipedia.org//wiki/ONE_Friday_Fights_13',
 'https://en.wikipedia.org//wiki/ONE_Friday_Fights_12',
 'https://en.wikipedia.org//wiki/ONE_Friday_Fights_18',
 'https://en.wikipedia.org//wiki/ONE_Friday_Fights_17',
 'https://en.wikipedia.org//wiki/ONE_Friday_Fights_8',
 'https://en.wikipedia.org//wiki/2020_in_ONE_Championship',
 'https://en.wikipedia.org//wiki/2018_in_ONE_Championship',
 'https://en.wikipedia.org//wiki/2022_in_ONE_Championship',
 'https://en.wikipedia.org//wiki/ONE_Friday_Fights_11',
 'https://en.wikipedia.org//wiki/2019_in_ONE_Championship']